In [1]:
import cv2;
import math;
import numpy as np;
from matplotlib import pyplot as plt
import os
def DarkChannel(im,sz):
    b,g,r = cv2.split(im)
    dc = cv2.min(cv2.min(r,g),b);
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT,(sz,sz))
    dark = cv2.erode(dc,kernel)
    if not os.path.exists('dark channel prior/data'): 
        os.makedirs('dark channel prior/data') 
    cv2.imwrite('dark channel prior/data/dark channel map dc.png', dc*255 )
    cv2.imwrite('dark channel prior/data/dark channel map dr.png', dark*255 )
#     cv2.imshow('dark channel',dark)
#     cv2.waitKey()
    return dark

def AtmLight(img,dark):
    [h,w] = img.shape[:2]
    imsz = h*w
    numpx = int(max(math.floor(imsz/1000),1))
    darkvec = dark.reshape(imsz,1);
    imvec = img.reshape(imsz,3);

    indices = darkvec.argsort();
    indices = indices[imsz-numpx::]
    atmsum = np.zeros([1,3])
    for ind in range(1,numpx):
        atmsum = atmsum + imvec[indices[ind]]
    A = atmsum / numpx;
    return A

def TransmissionEstimate(im,A,sz):
    omega = 0.95;
    im3 = np.empty(im.shape,im.dtype);

    for ind in range(0,3):
        im3[:,:,ind] = im[:,:,ind]/A[0,ind]

    transmission = 1 - omega*DarkChannel(im3,sz);
    cv2.imwrite('dark channel prior/data/TransmissionEstimate.png', transmission*255 )
#     cv2.imshow('TransmissionEstimate',transmission)
#     cv2.waitKey()
    return transmission

def Guidedfilter(im,p,r,eps):
    mean_I = cv2.boxFilter(im,cv2.CV_64F,(r,r))
    mean_p = cv2.boxFilter(p, cv2.CV_64F,(r,r))
    mean_Ip = cv2.boxFilter(im*p,cv2.CV_64F,(r,r))
    cov_Ip = mean_Ip - mean_I*mean_p

    mean_II = cv2.boxFilter(im*im,cv2.CV_64F,(r,r))
    var_I   = mean_II - mean_I*mean_I

    a = cov_Ip/(var_I + eps)
    b = mean_p - a*mean_I

    mean_a = cv2.boxFilter(a,cv2.CV_64F,(r,r))
    mean_b = cv2.boxFilter(b,cv2.CV_64F,(r,r))

    q = mean_a*im + mean_b;
    return q

def TransmissionRefine(im,et):
    gray = cv2.cvtColor(im,cv2.COLOR_BGR2GRAY);
    gray = np.float64(gray)/255;
    r = 60;
    eps = 0.0001;
    t = Guidedfilter(gray,et,r,eps)
    #t=et
    cv2.imwrite('dark channel prior/data/medium transmission.png', t*255 )
#     cv2.imshow('medium transmission',t)
#     cv2.waitKey()
    #t is the medium transmission describing 
    #the portion of the light that is not scattered and reaches the camera
    return t;

def Recover(im,t,A,tx = 0.1):
    res = np.empty(im.shape,im.dtype);
    t = cv2.max(t,tx);

    for ind in range(0,3):
        res[:,:,ind] = (im[:,:,ind]-A[0,ind])/t + A[0,ind]
#     cv2.imshow('recover',res)
#     cv2.waitKey()
    return res

In [8]:
nim= 13;keyword = 'haze'
for i in range(nim):
    inputImagePath = 'haze images/haze'+str(i)+'.jpg'
    name = 'haze'+str(i)+'.jpg'
    src = cv2.imread(inputImagePath)

    I = src.astype('float64')/255;

    dark = DarkChannel(I,15);
    A = AtmLight(I,dark)
    te = TransmissionEstimate(I,A,15)

    t = TransmissionRefine(src,te)
    d = np.exp(-t)
    plt.imsave('dark channel prior/data/depth map'+str(i)+'.png',d,cmap='Greys_r')
    J = Recover(I,t,A,0.1);
    if not os.path.exists('dark channel prior/dehaze result'): 
            os.makedirs('dark channel prior/dehaze result') 
    cv2.imwrite('dark channel prior/dehaze result/'+name+'.png', J*255 )
